In [61]:
import math
import pandas as pd

# Reference 

1. Association of Structural Engineers of the Philippines 2015 Edition, National Structural Code of the Philippines Volume 1, Quezon City, Philippines.
2. American Concrete Institute, ACI-440R-17

In [20]:
input = {
    "project_name" : "Makati Life Nurture Medical Center <i>Formerly Ospital ng Makati District 1</i>",
    "project_location" : "Malugay Street, Barangay Bel Air, Makati City ",
    "designer" : "Engr. Albert Pamonag, M.Eng",
    "l" : 7.32,
    "w" : 305,
    "d" : 546,
    "h" : 609.6,
    "fc" : 34.5,
    "fy" : 414,
    "Mn" : 361,
    "db" : 28.6,
    "Es" : 200,
    "exposure" : {
        "condition" : "interior",
        "fiber" : "carbon"
    },
    "frp" : {
        "tf" : 1.02,
        "ffu" : 621,
        "efu" : 0.015,
        "Ef" : 37000,
        "psi_f" : 0.85
    }
}

In [21]:
def eRF(exposure,type):
    data = {
        "interior" : {
            "carbon" : 0.95,
            "glass" : 0.75,
            "aramid" : 0.85
        },
        "exterior" : {
            "carbon" : 0.85,
            "glass" : 0.65,
            "aramid" : 0.75
        },
        "aggressive" : {
            "carbon" : 0.85,
            "glass" : 0.50,
            "aramid" : 0.70
            },
    }
    results = data[exposure][type]
    return results

### Computation the FRP system design material properties :

Design Ultimate Tensile Strength of FRP:
<center>$ f_{fu} = C_{E} f^{*}_{fu} $</center>

In [52]:
ffu = eRF(input['exposure']['condition'],input['exposure']['fiber'])*input['frp']['ffu']
print(str(round(ffu,3)) + " N/mm^2")

589.95 N/mm^2


Design Rupture Strain of FRP reinforcement attained at failure:
<center>$ \varepsilon_{fu} = C_{E} f^{*}_{fu} $</center>

In [53]:
efu = eRF(input['exposure']['condition'],input['exposure']['fiber'])*input['frp']['efu']
print(str(round(efu,3)) + " N/mm^2")

0.014 N/mm^2


In [54]:
def beta_1(fc): 
    ##NSCP 2015 422.2.2.4.3
    value = 1.05 - 0.05*(fc/1000)
    if(17 <= fc and fc <= 28):
        value = 0.85
    elif (28 <= fc and fc < 55):
        value = 0.85 - ((0.05*(fc-28))/7)
    else:
        value = 0.65

    return value
    
def areaCircle(db):
    pi = math.pi
    value = (pi/4)*(math.pow(db,2))
    return value
    
beta1 = beta_1(input['fc'])
ec = 4700*math.sqrt(input['fc'])
n_bot = 3
n_frp = 2
print(beta1)

area_steel = n_bot*areaCircle(input['db'])
af = n_frp*input["frp"]['tf']*input['w']

print(area_steel)
print(af)

0.8035714285714286
1927.2728451977305
622.2


$ \beta _{1} $

In [55]:
print(beta1)

0.8035714285714286


### Modulus of Elasticity of Concrete
<center> $ E_{c} = 4700 \sqrt{ f_{c} } $ </center>

In [56]:
print(ec)

27606.249292506214


## Properties of the concrete:

### Determine the existing state of strain on the soffit
<center>$ \epsilon_{bi} = \frac{ M _{DL} \times \left( d_{f} - kd \right) }{ I_{cr} \times E_{c} } $</center>

In [37]:
k = 0.334
    
Icr = 2.471e6 ##mm4 TODO double check this one
df = input['h']
d = input['d']
MDL = 97.6
ec_convert = ec/1000

ebi = ( MDL * (df - (k*d)) )  /( Icr * ec_convert ) 

## Determine the design strain of the FRP system
<center> $ \epsilon _{fd} = 0.41 \times \sqrt{ \frac{ f_{c}^{'} }{ 2 \times E_{f} \times t_{f} } } $ </center>

In [59]:
efd_init = input['fc']/(2*input["frp"]['tf']*input["frp"]['Ef'])
efd = 0.41*math.sqrt(efd_init)
print(efd)

0.008765515428730905


## Computation of the compression block

In [69]:
def cSolver():## //Numerical Solver
        
    ##--> Data for the tabulations
    epsilon_fe_arr = []
    espilon_c_arr = []
    eespilon_s_arr = []
    e_c_prime_arr = []
    beta_one_arr = []
    alpha_one_arr = []
    c_init_arr = []

    c_init = 0.20*input['d'] ##initialize depth
    i = 0
    
    while i <= 20:
       
        epsilon_fe = 0.003*( (input['h'] - c_init) /c_init) - ebi
        espilon_c = (0.009 + ebi)*(c_init/(input['h'] - c_init))
        espilon_s = (0.009 + ebi)*((input['d'] - c_init)/(input['h'] - c_init))

        fs = input['Es']*espilon_s
        fs = 414 ##TODO fix on the yields
            
        f_fe = input['frp']['Ef']*0.009
            
        e_c_prime = 1.7*input['fc']/ec
        beta_one = ((4*e_c_prime )- espilon_c)/((6*e_c_prime)-(2*espilon_c))
        alpha_one = ((3*e_c_prime*espilon_c) - math.pow(espilon_c,2))/(3*beta_one*math.pow(e_c_prime,2))    
        c_init = (area_steel*fs + af*f_fe)/(alpha_one*input['fc']*beta_one*input['w'])

        epsilon_fe_arr.append(epsilon_fe)
        espilon_c_arr.append(espilon_c)
        eespilon_s_arr.append(espilon_s)
        e_c_prime_arr.append(e_c_prime)
        beta_one_arr.append(beta_one)
        alpha_one_arr.append(alpha_one)
        c_init_arr.append(c_init)
    
        c = c_init
        i += 1
    results = {
        "value" : c,
        "epsilon_fe" : epsilon_fe_arr,
        "espilon_c" : espilon_c_arr,
        "espilon_s" : eespilon_s_arr,
        "e_c_prime" : e_c_prime_arr,
        "beta_one" : beta_one_arr,
        "alpha_one" : alpha_one_arr,
        "c_init" : c_init_arr,
    }
        
    return results

c_data = cSolver()
df = pd.DataFrame({
      'c (mm)': c_data['c_init'],
      'ε_fe': c_data['epsilon_fe'],    
      'ε_c': c_data['espilon_c'],
      'ε_s': c_data['espilon_s'],
      'ε_c\'': c_data['e_c_prime'],
      'β1': c_data['beta_one'],
      'α': c_data['alpha_one'],
      })
df

,c (mm),ε_fe,ε_c,ε_s,ε_c',β1,α
0,144.207618,0.013136,0.002097,0.008390,0.002125,0.748416,0.885018
1,127.904816,0.009070,0.002978,0.008298,0.002125,0.812854,0.918721
2,132.617164,0.010687,0.002552,0.008342,0.002125,0.777972,0.925806
3,130.765526,0.010179,0.002672,0.008330,0.002125,0.786996,0.928148
4,131.445102,0.010374,0.002625,0.008335,0.002125,0.783360,0.927636
5,131.188514,0.010302,0.002642,0.008333,0.002125,0.784680,0.927886
6,131.284414,0.010329,0.002636,0.008334,0.002125,0.784180,0.927800
7,131.248432,0.010319,0.002638,0.008333,0.002125,0.784367,0.927833
8,131.261913,0.010323,0.002637,0.008333,0.002125,0.784297,0.927821
9,131.256860,0.010321,0.002637,0.008333,0.002125,0.784323,0.927826


### Computation of Flexural Strength

#### Steel Contribution to Bending
<center> $ M_{ns} = A_{s} f_{s} \left( d - \frac{ \beta_{1} c }{2} \right) $ </center>

In [81]:
## Computation of Flexural Strength
n_length = len(c_data["beta_one"])
beta_one_final = c_data["beta_one"][n_length - 1]

##Computation Steel Contribution to Bending
fs = 414 ##--> to review soon
As = area_steel
Af = af
Mns = As*fs*(input['d'] - ((beta_one_final*c_data["value"])/2))
f_fe = input['frp']['Ef']*0.009

#### FRP Contribution to Bending
<center> $ M_{nf} = A_{s} f_{fe} \left( d - \frac{ \beta_{1} c }{2} \right) $ </center>

In [82]:
Mnf = Af*f_fe*(input["h"] - (beta_one_final*c_data["value"])/2)
print(Mnf/1e6)

# Computation of design flexural strength of the section

psi_f = 0.85 ##research 
phi_b = 0.90
phiMn = phi_b*(Mns + (psi_f*Mnf))

115.63958816829273


### Computation Design Flexural Strength of the Section
<center>$ \phi M_{n} = \phi \left[ M_{ns} + \psi_{f} M_{nf} \right] $</center>

In [84]:
psi_f = 0.85 ##research 
phi_b = 0.90
phiMn = phi_b*(Mns + (psi_f*Mnf))
print(phiMn/1e6)

443.5843648893501


### Calculate service stresses in reinforcing steel and the FRP

<center> $ k = \sqrt{ \left( \rho_{s} \frac{E_{s}}{E_{c}} + \rho_{f} \frac{E_{f}}{E_{c}} \right)^{2} + 2.0 \times \left[ \rho_{s} \frac{E_{s}}{E_{c}} + \rho_{f} \frac{E_{f}}{E_{c}} \times \left( \frac{d_{f}}{d} \right) \right] } - \left( \rho_{s} \frac{E_{s}}{E_{c}} + \rho_{f} \frac{E_{f}}{E_{c}} \right) $ </center>

In [88]:
def rho(As,b,d):
    value = As/(b*d)
    return value



rho_s = rho(area_steel,input["h"],input["w"])
rho_f = rho(af,input["h"],input["w"])

Es = 200
Ec = ec/1000
Ef = input['frp']['Ef']/1000

k1 = math.pow(rho_s*(Es/Ec) + rho_f*(Ef/Ec),2)
k2 = 2*(rho_s*(Es/Ec) + rho_f*(Ef/Ec)*(input['h']/input['d']))
k3 = rho_s*(Es/Ec) + rho_f*(Ef/Ec)

k = math.sqrt(k1 + k2) - k3
print(k)

0.32851365041305364


In [ ]:


#     report += `<h3>Step 8 - Calculate the stress level in the reinforcing steel and FRP </h3>`
    
#     var fs = input_data['Es']*data['espilon_s']
#     report += `<center> \$ f_{s} =  ${fs.toFixed(5)} \$ </center>`
#     fs = 414 //TODO fix on the yields
#     var f_fe = input_data['frp']['Ef']*0.009
#     report += `<center> \$ f_{fe} =  ${f_fe.toFixed(5)} \$ </center>`
#     data['fs'] = fs
#     data['f_fe'] = f_fe
#     return {
#         report
#     }

# }

In [ ]:
# const input_data = require('../data/test.json'); //insert data here
# module.exports = function (report, data) {

#     report += `<h3>Step 9 - Calculate the internal force resultants and check equilibrium</h3>`
#     var e_c_prime = 1.7*input_data['fc']/data['ec']

#     var beta_one = ((4*e_c_prime )- data['espilon_c'])/((6*e_c_prime)-(2*data['espilon_c']))
#     var alpha_one = ((3*e_c_prime*data['espilon_c']) - Math.pow(data['espilon_c'],2))/(3*beta_one*Math.pow(e_c_prime,2))
#     report += `<center> \$ \\epsilon_{c}^{'} =  ${e_c_prime.toFixed(5)} \$ </center>`
#     report += `<center> \$ \\beta_{1} =  ${beta_one.toFixed(5)} \$ </center>`
#     report += `<center> \$ \\alpha_{1} =  ${alpha_one.toFixed(5)} \$ </center>`

#     var c = (data['as']*data['fs'] + data['af'] *data['f_fe'])/(alpha_one*input_data['fc']*beta_one*input_data['w'])
#     report += `<center> \$ c =  ${c.toFixed(5)} \$ </center>`

#     return {
#         report
#     }

# }